<a href="https://colab.research.google.com/github/HasunJung/llm_langchain/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

chat bot

In [39]:
#최초 실행이라면
!pip install -q langchain langchain-google-genai pandas kor langchain-text-splitters sentence-transformers chromadb markdownify

import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ChatGoogleGenerativeAI 언어 모델을 "gemini-pro" 모델로 초기화합니다. 답변의 일관성을 위해 temperature는 0으로 설정
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

# Document Loaders

In [2]:
import ssl
import urllib.request
from IPython.display import Markdown
import json

url = 'https://lineagem.plaync.com/api/v2/lineagem/guide/%EC%B0%BD%EA%B3%A0%20%EC%8B%9C%EC%8A%A4%ED%85%9C'

# Set up SSL context to allow legacy TLS versions
ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT

response = urllib.request.urlopen(url, context=ctx)
guide_data = response.read().decode("utf-8")

guide_json = json.loads(guide_data)


In [38]:
#new dataFrame
paragraph_list = list()
paragraph_str = ""

idx = 0;
for tmp in guide_json["paragraphList"]:
  paragraph_list.append(guide_json['paragraphList'][tmp]['name']+":"+ guide_json['paragraphList'][tmp]['content'])
  paragraph_str+=guide_json['paragraphList'][tmp]['name']+":"+ guide_json['paragraphList'][tmp]['content']
  idx=+1


import pandas as pd

df = pd.DataFrame({
    'name' : guide_json['name'],
    'summary' : guide_json['summary'],
    'paragraphList' : paragraph_list,
    'paragraph' : paragraph_str
})


from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column = 'paragraph')
docs = loader.load()

print(docs[0].page_content)


소개:<p>창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.</p><p>&lt;리니지M&gt;에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.</p><p>무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.</p><p><strong>&lt;무게 게이지에 따른 패널티&gt;</strong></p><p>* 50% 이상 : HP 및 MP 자동 회복 불가</p><p>* 83% 이상 : 공격 및 마법 사용 불가</p><p><br></p>창고지기:<p>각 마을에 위치한 창고지기 NPC를 찾아가면 창고를 이용할 수 있습니다. 대부분 창고지기 NPC는 마을 중심부에 위치해 있으며, 마을 지도에 상자 모양의 아이콘으로 표시됩니다.</p><p><img style="width: 300px;" class="fr-fic fr-dib" src="https://imgfiles-cdn.plaync.com/file/LineageM/download/20220110202731-D6HdPr98oizxe1hTOIIR0-v4"></p><p><br></p>창고 UI:<p><img style="width: 300px;" class="fr-fic fr-dib" src="https://imgfiles-cdn.plaync.com/file/LineageM/download/20220110202815-cSHmNzD5q95vFGQFaxYa0-v4"></p><p>① 소지품 : 소지한 아이템 정보를 확인할 수 있습니다.</p><p>② 창고 : 창고에 보관된 아이템 정보를 확인할 수 있습니다.</p><p>③ 슬롯 모드 변경 버튼 : 리스트 모드 또는 인벤토리 모드 중 한가지를 선택할 수 있습니다.</p><p>④ 장바구니 : 소지품 또는 창고에서 아이템을 선택한 다음 수

# RecursiveCharacterTextSplitter

In [41]:
from kor.documents.html import MarkdownifyHTMLProcessor
md = MarkdownifyHTMLProcessor().process(docs[0])


from langchain_text_splitters import RecursiveCharacterTextSplitter
split_docs = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0).split_documents([md])

print(split_docs)
print(split_docs[0].page_content)


[Document(page_content='소개:창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.\n\n<리니지M>에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.\n\n무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.\n\n**<무게 게이지에 따른 패널티>**\n\n\\* 50% 이상 : HP 및 MP 자동 회복 불가\n\n\\* 83% 이상 : 공격 및 마법 사용 불가', metadata={'name': '창고 시스템', 'summary': '소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간 입니다.', 'paragraphList': '소개:<p>창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.</p><p>&lt;리니지M&gt;에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.</p><p>무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.</p><p><strong>&lt;무게 게이지에 따른 패널티&gt;</strong></p><p>* 50% 이상 : HP 및 MP 자동 회복 불가</p><p>* 83% 이상 : 공격 및 마법 사용 불가</p><p><br></p>'}), Document(page_content='창고지기:각 마을에 위치한 창고지기 NPC를 찾아가면 창고를 이용할 수 있습니다. 대부분 창고지기 NPC는 마을 중심부에 위치해 있으며, 마을 지도에 상자 모양의 아이콘으로 표시됩니다.\n\n창고 UI:① 소지품 : 소지한 아이템 정보를 확인할 

In [187]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


# VectorDB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=split_docs, embedding=ko_embedding)

In [ ]:
PromptTemplate(llm, "캐릭터 무게 게이지 패널티 정보 알려줘")

In [192]:
similar_docs = vectordb.similarity_search("캐릭터 무게 게이지 패널티 정보 알려줘")
print(similar_docs[0].page_content)

창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.

<리니지M>에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.

무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.

**<무게 게이지에 따른 패널티>**

\* 50% 이상 : HP 및 MP 자동 회복 불가

\* 83% 이상 : 공격 및 마법 사용 불가
